<a href="https://colab.research.google.com/github/refaaksi/ML-dicoding/blob/main/ML_dicoding_Submission_Refa_Rupaksi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import (
    BatchNormalization,
    Conv2D,
    Dense,
    Flatten,
    Input,
    MaxPooling2D,
    GlobalAveragePooling2D, 
    GlobalMaxPooling2D,
    Activation,
    Dropout
)
from tensorflow.keras import Model
import zipfile, os

In [ ]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 

print('Available GPUs: ', len(tf.config.list_logical_devices('GPU')))

In [ ]:
!wget command: --no-check-certificate \
  https://dicodingacademy.blob.core.windows.net/picodiploma/ml_pemula_academy/rockpaperscissors.zip \
  -O /tmp/rockpaperscissors.zip

In [ ]:
# melakukan ekstraksi pada file zip
local_zip = '/tmp/rockpaperscissors.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

base_dir = '/tmp/rockpaperscissors/rps-cv-images/'
os.listdir(base_dir)

In [ ]:
os.chdir(base_dir)
with open('README_rpc-cv-images.txt', 'r') as f:
  r = f.read(-1)

print(r)

In [ ]:
BATCH_SIZE=32

def normalize_img(image,label):
  return tf.cast(image, tf.float32)/255, label

ds_train = tf.keras.preprocessing.image_dataset_from_directory(
    base_dir,
    labels = 'inferred',
    label_mode = "categorical",
    batch_size = BATCH_SIZE,
    color_mode = 'rgb',
    image_size = (300,200),
    seed = 0,
    shuffle = True,
    validation_split = 0.4,
    subset = "training")

ds_val = tf.keras.preprocessing.image_dataset_from_directory(
    base_dir,
    labels = 'inferred',
    label_mode = "categorical",
    batch_size = BATCH_SIZE,
    color_mode = 'rgb',
    image_size = (300,200),
    seed = 0,
    shuffle = True,
    validation_split = 0.4,
    subset = "validation")

ds_train = ds_train.map(normalize_img)
ds_val = ds_val.map(normalize_img)

In [ ]:
data_augmentation = tf.keras.models.Sequential(
    [
        tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal"),
        tf.keras.layers.experimental.preprocessing.RandomRotation(0.1),
    ]
)

In [ ]:
def conv_block(units, activation='relu', block=1, layer=1):

    def layer_wrapper(inp):
        x = Conv2D(units, (3, 3), name='block{}_conv{}'.format(block, layer))(inp)
        # x = BatchNormalization(name='block{}_bn{}'.format(block, layer))(x)
        x = Activation(activation, name='block{}_act{}'.format(block, layer))(x)
        return x

    return layer_wrapper

def dense_block(units, dropout=0.2, activation='relu', name='fc1'):

    def layer_wrapper(inp):
        x = Dense(units, name=name)(inp)
        #x = BatchNormalization(name='{}_bn'.format(name))(x)
        x = Activation(activation, name='{}_act'.format(name))(x)
        x = Dropout(dropout, name='{}_dropout'.format(name))(x)
        return x

    return layer_wrapper

def make_model(input_shape, classes, dropout=0.3, activation='relu'):
    inputs = Input(shape=input_shape)
    # Image augmentation block
    X_input = data_augmentation(inputs)

    x = conv_block(32, activation=activation, block=1, layer=1)(X_input)
    x = MaxPooling2D((2, 2), name='block1_pool')(x)

    # Block 2
    x = conv_block(64, activation=activation, block=2, layer=1)(x)
    x = MaxPooling2D((2, 2), name='block2_pool')(x)

    # Block 3
    x = conv_block(128, activation=activation, block=3, layer=1)(x)
    x = MaxPooling2D((2, 2), name='block3_pool')(x)

    # # Block 4
    x = conv_block(256, activation=activation, block=4, layer=1)(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)

    x = conv_block(512, activation=activation, block=5, layer=1)(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool')(x)

    # Flatten
    x = Flatten()(x)

    x = dense_block(512, dropout=dropout, activation=activation, name='fc1')(x)
    
    # Classification block    
    outputs = Dense(classes, activation='softmax', name='predictions')(x)

    return Model(inputs, outputs)

In [ ]:
model = make_model(input_shape = (300, 200, 3), classes = 3)
model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
history = model.fit(
      ds_train,
      batch_size=BATCH_SIZE,
      epochs=25,
      validation_data=ds_val,
      verbose=2)

In [ ]:
#eval = model.evaluate(ds_val, verbose=1)

In [ ]:
from matplotlib import pyplot as plt
#loss train & validation
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Loss Plot')
plt.ylabel('Value')
plt.xlabel('Epoch')
plt.legend(loc="upper right")
plt.show()

In [ ]:
#accuracy train & validation
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Accuracy Plot')
plt.ylabel('Value')
plt.xlabel('Epoch')
plt.legend(loc="lower right")
plt.show()